<a href="https://colab.research.google.com/github/96msnbykym/BERT_sentiment_tweet/blob/main/BERT_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mecab-python3

In [ ]:
pip install fugashi  # tokenizerが内部で使用

In [ ]:
pip install ipadic  # tokenizerが内部で使用

In [ ]:
pip install transformers

In [8]:
import pandas as pd
import torch
from torch.optim import AdamW
from transformers import pipeline, AutoModelForSequenceClassification, BertJapaneseTokenizer

nlp = pipeline("sentiment-analysis",model=model,tokenizer=tokenizer)

In [9]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('drive/My Drive/data/sugakubunka.csv')
df.dropna(inplace=True)
df = df.drop(["Unnamed: 0","text_cleaned"],axis=1)
df["label"] = df["label"].astype(int)
df = df.replace({0:'NEGATIVE', 1:'POSITIVE'})
df

Mounted at /content/drive


,text,label
0,サンプルコード見ながらバグってる,NEGATIVE
1,python多めの後にR触るとdata.trainとか書いてあるとき頭がバグる,NEGATIVE
7,痩せる,POSITIVE
8,ホームページの写真、新しくしてもらったんだけど顔丸すぎて萎えた,NEGATIVE
9,はい！再来週やります！\n無料なのでお気軽にご参加ください😊 https://t.co/Vf...,POSITIVE
...,...,...
172,肉眼ならいい感じに見えたのになー,POSITIVE
173,都会の月食(うるとらわかりにくい) https://t.co/5wIeonJexx,NEGATIVE
174,復習します！,POSITIVE
175,講師勉強会、今日はとてもむじゅかしぃでした\n(ミュート外れてるのに気付かずに独り言聞かれた人),POSITIVE


In [10]:
train_docs = df["text"].tolist()
train_labels = df["label"].tolist()

In [ ]:
pip install fugashi[unidic-lite]

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

model_name = "cl-tohoku/bert-large-japanese"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, id2label=id2label, label2id=label2id)
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [12]:
encodings = tokenizer(train_docs, return_tensors='pt', padding=True, truncation=True, max_length=128)
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']

In [ ]:
# Fine-tuning in native PyTorch

# the AdamW() optimizer which implements gradient bias correction as well as weight decay.
optimizer = AdamW(model.parameters(), lr=1e-5)

labels = [label2id[label] for label in train_labels]
labels = torch.tensor(labels).unsqueeze(0)
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
loss = outputs.loss
loss.backward()
optimizer.step()